<a href="https://colab.research.google.com/github/lululu-mia/Data-Science/blob/main/Homework/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW4

## 1050. Actors and Directors Who Cooperated At Least Three Times

MySQL

In [ ]:
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING count(*) >= 3

Pandas

In [ ]:
import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    grouped = (
        actor_director
        .groupby(['actor_id', 'director_id'])
        .size()
        .reset_index(name='count')
    )
    result = grouped[grouped['count'] >= 3][['actor_id', 'director_id']]
    return result

## 1667. Fix Names in a Table

MySQL

In [ ]:
SELECT
    user_id,
    CONCAT(
        UPPER(LEFT(name, 1)),
        LOWER(SUBSTRING(name, 2))
    ) AS name
FROM
    Users
ORDER BY
    user_id;

Pandas

In [ ]:
import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users['name'] = users['name'].str.capitalize()
    users = users.sort_values('user_id').reset_index(drop=True)
    return users

## 175. Combine Two Tables

MySQL

In [ ]:
SELECT
    p.firstName,
    p.lastName,
    a.city,
    a.state
FROM
    Person p
LEFT JOIN
    Address a
ON
    p.personId = a.personId;

Pandas

In [ ]:
import pandas as pd

def person_address(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:

    result = pd.merge(
        person,
        address,
        on='personId',
        how='left'
    )

    result = result[['firstName', 'lastName', 'city', 'state']]

    return result

## 176. Second Highest Salary

MySQL

In [ ]:
SELECT
    (
        SELECT DISTINCT salary
        FROM Employee
        ORDER BY salary DESC
        LIMIT 1 OFFSET 1
    ) AS SecondHighestSalary;

Pandas

In [ ]:
import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)

    if len(salaries) >= 2:
        second_highest = salaries.iloc[1]
    else:
        second_highest = None

    return pd.DataFrame({'SecondHighestSalary': [second_highest]})

## 1327. List the Products Ordered in a Period

MySQL

In [ ]:
SELECT
    p.product_name,
    SUM(o.unit) AS unit
FROM
    Products p
JOIN
    Orders o
ON
    p.product_id = o.product_id
WHERE
    YEAR(o.order_date) = 2020
    AND MONTH(o.order_date) = 2
GROUP BY
    p.product_id, p.product_name
HAVING
    SUM(o.unit) >= 100;

Pandas

In [ ]:
import pandas as pd

def february_orders(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    orders['order_date'] = pd.to_datetime(orders['order_date'])

    feb_orders = orders[
        (orders['order_date'].dt.year == 2020) &
        (orders['order_date'].dt.month == 2)
    ]

    feb_sum = feb_orders.groupby('product_id', as_index=False)['unit'].sum()

    feb_sum = feb_sum[feb_sum['unit'] >= 100]

    result = pd.merge(feb_sum, products, on='product_id', how='inner')

    result = result[['product_name', 'unit']]

    return result

## 1378. Replace Employee ID With The Unique Identifier

MySQL

In [ ]:
SELECT
    e.unique_id,
    p.name
FROM
    Employees p
LEFT JOIN
    EmployeeUNI e
ON
    p.id = e.id;

Pandas

In [ ]:
import pandas as pd

def employee_unique_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    result = pd.merge(
        employees,
        employee_uni,
        on='id',
        how='left'
    )

    result = result[['unique_id', 'name']]

    return result

## 550. Game Play Analysis IV

MySQL

In [ ]:
SELECT
    ROUND(
        COUNT(DISTINCT a1.player_id) / COUNT(DISTINCT a2.player_id),
        2
    ) AS fraction
FROM
    Activity a1
JOIN
    (
        SELECT player_id, MIN(event_date) AS first_login
        FROM Activity
        GROUP BY player_id
    ) a2
ON
    a1.player_id = a2.player_id
    AND DATEDIFF(a1.event_date, a2.first_login) = 1;

Pandas

In [ ]:
import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    first_login = (
        activity.groupby('player_id')['event_date']
        .min()
        .reset_index(name='first_login')
    )

    merged = pd.merge(activity, first_login, on='player_id')

    active_next_day = merged[
        (merged['event_date'] - merged['first_login']).dt.days == 1
    ]['player_id'].nunique()

    total_players = first_login['player_id'].nunique()

    fraction = round(active_next_day / total_players, 2)

    return pd.DataFrame({'fraction': [fraction]})

## 1075. Project Employees I

MySQL

In [ ]:
SELECT
    p.project_id,
    ROUND(AVG(e.experience_years), 2) AS average_years
FROM
    Project p
JOIN
    Employee e
ON
    p.employee_id = e.employee_id
GROUP BY
    p.project_id;

Pandas

In [ ]:
import pandas as pd

def project_employees(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(project, employee, on='employee_id')

    result = (
        merged.groupby('project_id', as_index=False)['experience_years']
        .mean()
        .round(2)
    )

    result.rename(columns={'experience_years': 'average_years'}, inplace=True)

    return result

## 185. Department Top Three Salaries

MySQL

In [ ]:
SELECT
    d.name AS Department,
    e.name AS Employee,
    e.salary AS Salary
FROM (
    SELECT
        *,
        DENSE_RANK() OVER (PARTITION BY departmentId ORDER BY salary DESC) AS rnk
    FROM Employee
) e
JOIN Department d
ON e.departmentId = d.id
WHERE e.rnk <= 3;

Pandas

In [ ]:
import pandas as pd

def department_top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(employee, department, left_on='departmentId', right_on='id', suffixes=('_emp', '_dept'))

    merged['rnk'] = merged.groupby('departmentId')['salary'].rank(method='dense', ascending=False)

    top3 = merged[merged['rnk'] <= 3]

    result = top3[['name_dept', 'name_emp', 'salary']].rename(
        columns={'name_dept': 'Department', 'name_emp': 'Employee', 'salary': 'Salary'}
    )

    return result